In [48]:
import json
import pandas as pd
import requests

In [49]:
stock = input("enter the stock ticker symbol: ")
apikey = "Ii1JSP9XiA9Xk46itzFe43U7Z64InunH4d5skuC3" 
url = "https://yfapi.net/v6/finance/quote"
querystring = {"symbols": stock}
headers = {'x-api-key': apikey}

In [50]:
try:
    response = requests.get(url, headers=headers, params=querystring)
    response.raise_for_status()
    stock_json = response.json()

    if 'quoteResponse' in stock_json and len(stock_json['quoteResponse']['result']) > 0:
        
        stock_data = stock_json['quoteResponse']['result'][0]
        ticker = stock_data.get('symbol', 'N/A')
        full_name = stock_data.get('longName', 'N/A')
        current_price = stock_data.get('regularMarketPrice', 'N/A')
        
        finance_url = "https://yfapi.net/v11/finance/quoteSummary/" + str(ticker)
        finance_response = requests.get(finance_url, headers=headers, params={"modules": "financialData"})
        finance_response.raise_for_status()
        financial_data = finance_response.json().get('quoteSummary', {}).get('result', [{}])[0].get('financialData', {})
        target_mean_price = financial_data.get('targetMeanPrice', {}).get('raw', 'N/A')
        
        week_high = stock_data.get('fiftyTwoWeekHigh', 'N/A')
        week_low = stock_data.get('fiftyTwoWeekLow', 'N/A')
        
        print("ticker: " + ticker + "\n" +
              "full name: " + full_name + "\n" +
              "current market price: $" + str(current_price) + "\n" +
              "target mean price: $" + str(target_mean_price) + "\n" +
              "52 week high: $" + str(week_high) + "\n" +
              "52 week low: $" + str(week_low))
        
        # save data frame
        data = {"ticker": [ticker], "full name": [full_name], "current price": [current_price], "target mean price": [target_mean_price], "52 week high": [week_high], "52 week low": [week_low]}
        
        df = pd.DataFrame(data)
        df.to_csv("stock.csv", index=False)
        print("\ndata saved to stock.csv.")
        
    else:
        print("cannot get info for '" + str(stock) + "' and thus no csv created")
    
    # 5 trending 
    trending_url = "https://yfapi.net/v1/finance/trending/US"
    trending_response = requests.get(trending_url, headers=headers)
    trending_response.raise_for_status()
    trending_json = trending_response.json()

    if 'finance' in trending_json and 'result' in trending_json['finance'] and len(trending_json['finance']['result']) > 0:
        trending_stocks = trending_json['finance']['result'][0]['quotes']
        print("\ntrending stocks:")
        trending_symbols = []
        count = 1
        for stock in trending_stocks[:5]:
            print(str(count) + ": " + str(stock['symbol']))
            count += 1
            trending_symbols.append(stock['symbol'])
    else:
        print("no trending stock")
 
except (requests.exceptions.HTTPError, Exception) as error:
    print("an error: " + str(error))

ticker: GOOG
full name: Alphabet Inc.
current market price: $165.29
target mean price: $198.92
52 week high: $193.31
52 week low: $121.46

data saved to stock.csv.

trending stocks:
1: AMSC
2: PWR
3: VRT
4: BBD
5: SNAP
